# "US Presidential Candidate Tweets"
> "A visualization of recent tweets from candidates in the 2020 US Presidential Election"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Data Visualization]
- hide: false
- search_exclude: true

In [1]:
#collapse-hide
from textblob import TextBlob
import altair as alt
import pandas as pd

# Link: https://joshkraft.github.io/joshkraft-blog/2020/10/21/Candidate-Tweets.html

In [2]:
#collapse-hide

def read_tweet_data(username, date):
  url = "https://raw.githubusercontent.com/joshkraft/daily-candidate-tweets/main/data/" + str(username) + "/" + str(date) + ".csv"
  df = pd.read_csv(url)
  return df

biden_tweets = read_tweet_data("JoeBiden", "2020-10-20")
trump_tweets = read_tweet_data("realDonaldTrump", "2020-10-20")

df = biden_tweets.append(trump_tweets)


new_biden_tweets = read_tweet_data("JoeBiden", "2020-10-21")
new_trump_tweets = read_tweet_data("realDonaldTrump", "2020-10-21")

df = df.append(new_biden_tweets)
df = df.append(new_trump_tweets)

df['sentiment'] = df['tweet_text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
df['subjectivity'] = df['tweet_text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)

In [3]:
chart = alt.Chart(df).mark_circle().encode(
    x='created_at',
    y='sentiment',
    size='retweets',
    color='username',
    tooltip=['tweet_text']
    ).interactive()

chart.properties(
    width=800,
    height=800
)

chart

alt.Chart(...)